In [1]:
import pandas as pd
import requests
import openai
import os
import sys
import plotly.express as px
import json
from typing import List
from pydantic import BaseModel, Field
sys.path.append("../")
from dotenv import load_dotenv
from finance_functions import *

In [2]:
# Load the environment variables from .env file
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
import langchain
print(langchain.__version__)

0.3.24


In [5]:
from langchain_core.tools import tool

In [6]:
import yfinance
yfinance.__version__

'0.2.59'

In [7]:
get_ticker.invoke("Unicredit")

'UCG.MI'

In [8]:
from langchain_core.messages import HumanMessage

In [9]:
query = "What are the financial of Apple?"

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-nano-2025-04-14", model_provider="openai", temperature=0)

In [11]:
tools = [get_ticker, get_stock_prices, get_financials, get_multiple_financials, get_institutional_holders,]

In [12]:
llm_with_tools = llm.bind_tools(tools)

In [13]:
ai_msg = llm_with_tools.invoke(query)

In [14]:
messages = [query, ai_msg]

In [15]:
print(ai_msg.tool_calls)

[{'name': 'get_ticker', 'args': {'company_name': 'Apple'}, 'id': 'call_CgonYAmP5yhoEKbxOF7DGF3J', 'type': 'tool_call'}]


In [16]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_ticker": get_ticker, "get_financials": get_financials,
                     "get_institutional_holders": get_institutional_holders, "get_multiple_financials": get_multiple_financials,
                     "get_stock_prices": get_stock_prices,
                     }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
messages

['What are the financial of Apple?',
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CgonYAmP5yhoEKbxOF7DGF3J', 'function': {'arguments': '{"company_name":"Apple"}', 'name': 'get_ticker'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 418, 'total_tokens': 435, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BVdnlFXytKU2Ex7OYuxmCCwUh4Gzu', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cce3e2e2-038f-48c3-bb28-d3907dfa2c8e-0', tool_calls=[{'name': 'get_ticker', 'args': {'company_name': 'Apple'}, 'id': 'call_CgonYAmP5yhoEKbxOF7DGF3J', 'type': 'tool_call'}], usage_metadata={'input_tokens': 418, 'output_tokens': 17, 'total_tokens': 435, 'i

In [ ]:
llm_with_tools.invoke(messages)

AIMessage(content='I have retrieved the financial information for Apple. Would you like a detailed summary or specific financial metrics?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 800, 'total_tokens': 822, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQc2SaIapvuVGsAXr4SOhu1yIkwnZ', 'finish_reason': 'stop', 'logprobs': None}, id='run-767e0830-0f3c-4e0d-b3ce-1e25f44d40d6-0', usage_metadata={'input_tokens': 800, 'output_tokens': 22, 'total_tokens': 822, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
print(messages[-1])

content='Saved the information in ../data/aapl_financials.csv, this is a preview \n|                                                     |   2024-09-30 00:00:00 |   2023-09-30 00:00:00 |   2022-09-30 00:00:00 |   2021-09-30 00:00:00 |   2020-09-30 00:00:00 |\n|:----------------------------------------------------|----------------------:|----------------------:|----------------------:|----------------------:|----------------------:|\n| TaxEffectOfUnusualItems                             |           0           |            0          |           0           |           0           |                   nan |\n| TaxRateForCalcs                                     |           0.241       |            0.147      |           0.162       |           0.133       |                   nan |\n| NormalizedEBITDA                                    |           1.34661e+11 |            1.2582e+11 |           1.30541e+11 |           1.23136e+11 |                   nan |\n| NetIncomeFromContinuingOperati

In [ ]:
messages.append("Give me Unicredit ticker")

In [17]:
ai_msg = llm_with_tools.invoke(messages)

In [18]:
ai_msg.tool_calls

[{'name': 'get_financials',
  'args': {'ticker': 'AAPL'},
  'id': 'call_YY6tRyACZQwHThksbkEToZ9j',
  'type': 'tool_call'}]